In [4]:
!pip install gcloud

In [5]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=DonDbfOmNqtj7KaguqbZqiCRKXRjYB&prompt=consent&token_usage=remote&access_type=offline&code_challenge=qQxJiaaZXsDqtAx2tPj6rhl7TbmsiApqirZSB3GvNuU&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AcvDMrA2lpXlJ5x2p13T9FdzsEs4b1z4cdpo-X245hKBnFMWcgzjk-au1lQRhYv6B_V3hQ

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [6]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re

In [7]:
client = bigquery.Client(project='repositoriodedadosgpsp')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [10]:
# Definir a consulta SQL
query = """
SELECT ano, 'publico' as flag_publico_privado,variavel,categoria, media_remuneracao FROM `repositoriodedadosgpsp.perfil_remuneracao.RAIS_remuneracao_vinculos_publicos_v2` union all SELECT ano, flag_publico_privado,variavel, categoria,media_remuneracao FROM `repositoriodedadosgpsp.perfil_remuneracao.RAIS_remuneracao_vinculos_privados_v2`
"""



# Executar a consulta e salvar os resultados em um DataFrame do pandas
df = client.query(query).to_dataframe()

# Exibir os primeiros registros do DataFrame
print(df.head())

    ano flag_publico_privado      variavel categoria  media_remuneracao
0  2022              privado  tipo_vinculo       CLT        3243.281870
1  2020              privado  tipo_vinculo       CLT        2386.224935
2  2003              privado  tipo_vinculo       CLT         804.920642
3  2001              privado  tipo_vinculo       CLT         699.824617
4  2010              privado  tipo_vinculo       CLT        1334.494196


In [13]:
query = "SELECT ano, 'publico' as flag_publico_privado,variavel,categoria, media_remuneracao FROM `repositoriodedadosgpsp.Datalake.RAIS_remuneracao_vinculos_publicos` union all SELECT ano, flag_publico_privado,variavel, categoria,media_remuneracao FROM `repositoriodedadosgpsp.Datalake.RAIS_remuneracao_vinculos_privados`"

In [14]:
df

,ano,flag_publico_privado,variavel,categoria,media_remuneracao
0,2022,privado,tipo_vinculo,CLT,3243.281870
1,2020,privado,tipo_vinculo,CLT,2386.224935
2,2003,privado,tipo_vinculo,CLT,804.920642
3,2001,privado,tipo_vinculo,CLT,699.824617
4,2010,privado,tipo_vinculo,CLT,1334.494196
...,...,...,...,...,...
1019,2020,publico,grau_instrucao,Até Ensino Superior Completo,6453.953895
1020,2009,publico,grau_instrucao,Até Ensino Superior Completo,3464.451507
1021,2017,publico,grau_instrucao,Até Ensino Superior Completo,5976.134972
1022,2011,publico,grau_instrucao,Até Ensino Superior Completo,4079.065605


In [15]:
## O privado vem com a série inteira, vou consertar essa parte ao consertar a série

df = df[df['ano']>2002]


In [16]:
df=df[df['variavel']!='tipo_vinculo']

In [17]:
df['ano'].unique()

<IntegerArray>
[2006, 2014, 2017, 2003, 2010, 2019, 2008, 2018, 2015, 2013, 2022, 2007, 2021,
 2012, 2004, 2009, 2016, 2020, 2005, 2011]
Length: 20, dtype: Int64

In [18]:
ipca = pd.read_csv("ipca.csv",sep=";",decimal=",")

In [19]:
ipca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Ano                     28 non-null     int64  
 1   IPCA acumulado dez/ano  28 non-null     float64
 2   It                      28 non-null     float64
 3   Fator de correcao       28 non-null     float64
dtypes: float64(3), int64(1)
memory usage: 1.0 KB


In [20]:
df1= df.merge(ipca[['Ano','Fator de correcao']], right_on='Ano',left_on='ano')

In [21]:
df1

,ano,flag_publico_privado,variavel,categoria,media_remuneracao,Ano,Fator de correcao
0,2006,privado,nivel_governo,Outros,973.471784,2006,2.476076
1,2006,privado,poderes,Outros,973.471784,2006,2.476076
2,2006,privado,genero,Feminino,821.379105,2006,2.476076
3,2006,privado,genero,Masculino,1057.065237,2006,2.476076
4,2006,privado,grau_instrucao,Até Mestrado,3466.256477,2006,2.476076
...,...,...,...,...,...,...,...
467,2011,publico,grau_instrucao,Até Mestrado,5426.678002,2011,1.902416
468,2011,publico,grau_instrucao,Até Doutorado,9670.870834,2011,1.902416
469,2011,publico,grau_instrucao,Até Fundamental,1323.279355,2011,1.902416
470,2011,publico,grau_instrucao,Até Ensino Médio,1899.718666,2011,1.902416


In [22]:
df1['media_remuneracao_ajustada']= df1['media_remuneracao']* df1['Fator de correcao']

In [23]:
df1

,ano,flag_publico_privado,variavel,categoria,media_remuneracao,Ano,Fator de correcao,media_remuneracao_ajustada
0,2006,privado,nivel_governo,Outros,973.471784,2006,2.476076,2410.390232
1,2006,privado,poderes,Outros,973.471784,2006,2.476076,2410.390232
2,2006,privado,genero,Feminino,821.379105,2006,2.476076,2033.797183
3,2006,privado,genero,Masculino,1057.065237,2006,2.476076,2617.373985
4,2006,privado,grau_instrucao,Até Mestrado,3466.256477,2006,2.476076,8582.714868
...,...,...,...,...,...,...,...,...
467,2011,publico,grau_instrucao,Até Mestrado,5426.678002,2011,1.902416,10323.799010
468,2011,publico,grau_instrucao,Até Doutorado,9670.870834,2011,1.902416,18398.019321
469,2011,publico,grau_instrucao,Até Fundamental,1323.279355,2011,1.902416,2517.427805
470,2011,publico,grau_instrucao,Até Ensino Médio,1899.718666,2011,1.902416,3614.055168


In [25]:
df1.columns

Index(['ano', 'flag_publico_privado', 'variavel', 'categoria',
       'media_remuneracao', 'Ano', 'Fator de correcao',
       'media_remuneracao_ajustada'],
      dtype='object')

In [26]:
df1=df1.rename(columns={'Fator de correcao':'fator_correcao'})

In [27]:
df1 = df1[['ano','flag_publico_privado', 'variavel', 'categoria', 'media_remuneracao',
       'fator_correcao', 'media_remuneracao_ajustada']]

In [28]:
df1['variavel'].unique()

array(['nivel_governo', 'poderes', 'genero', 'grau_instrucao'],
      dtype=object)

In [29]:
df1['flag_publico_privado']=np.where(df1['flag_publico_privado']=="privado","Privado",df1['flag_publico_privado'])
df1['flag_publico_privado']=np.where(df1['flag_publico_privado']=="publico","Público",df1['flag_publico_privado'])
df1

,ano,flag_publico_privado,variavel,categoria,media_remuneracao,fator_correcao,media_remuneracao_ajustada
0,2006,Privado,nivel_governo,Outros,973.471784,2.476076,2410.390232
1,2006,Privado,poderes,Outros,973.471784,2.476076,2410.390232
2,2006,Privado,genero,Feminino,821.379105,2.476076,2033.797183
3,2006,Privado,genero,Masculino,1057.065237,2.476076,2617.373985
4,2006,Privado,grau_instrucao,Até Mestrado,3466.256477,2.476076,8582.714868
...,...,...,...,...,...,...,...
467,2011,Público,grau_instrucao,Até Mestrado,5426.678002,1.902416,10323.799010
468,2011,Público,grau_instrucao,Até Doutorado,9670.870834,1.902416,18398.019321
469,2011,Público,grau_instrucao,Até Fundamental,1323.279355,1.902416,2517.427805
470,2011,Público,grau_instrucao,Até Ensino Médio,1899.718666,1.902416,3614.055168


In [30]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência'),
bigquery.SchemaField('flag_publico_privado','STRING',description='Se se refere ao setor publico ou privado'),
 bigquery.SchemaField('variavel','STRING',description='De qual variável aquela remuneração se refere'),
 bigquery.SchemaField('categoria','STRING',description='Categoria dentro daquela variável'),
 bigquery.SchemaField('media_remuneracao','FLOAT',description='Valor nominal da média de remuneração daquele ano'),
bigquery.SchemaField('fator_correcao','FLOAT',description='índice para correção monetária baseando-se no IPCA para dezembro de 2022'),
bigquery.SchemaField('media_remuneracao_ajustada','FLOAT',description='Valor nominal da média de remuneração daquele ano ajustado para dezembro de 2022')
 ]


In [31]:
client = bigquery.Client(project='repositoriodedadosgpsp')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [32]:
dataset_ref = client.dataset('perfil_remuneracao')

In [33]:
table_ref = dataset_ref.table('RAIS_remuneracao_vinculos_publicos_privados_v4')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df1, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=9f26e561-6739-4607-9869-541f729cc2df>

In [34]:
#para excluir tabelas
#client.delete_table('perfil_remuneracao.RAIS_remuneracao_vinculos_publicos_privados', not_found_ok=True)